In [ ]:
import pandas as pd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import math


%matplotlib inline

In [ ]:
aw_custs = pd.read_csv('AdvWorksCusts.csv')
aw_aveMonth = pd.read_csv('AW_AveMonthSpend.csv')
aw_bike=pd.read_csv('AW_BikeBuyer.csv')

In [ ]:
age=[]
for x in aw_custs.BirthDate:
    age.append(1998-int(x[:4]))
aw_custs['Age']=age
aw_custs.head().T

In [ ]:
aw_custs.drop_duplicates(subset='CustomerID', keep='last', inplace=True)
aw_aveMonth.drop_duplicates(subset='CustomerID', keep='last', inplace=True)
aw_bike.drop_duplicates(subset='CustomerID', keep='last', inplace=True)

In [ ]:
features=aw_custs['CountryRegionName']
enc=preprocessing.LabelEncoder()
enc.fit(features)
features=enc.transform(features)
ohe=preprocessing.OneHotEncoder()
encoded=ohe.fit(features.reshape(-1,1))
features = encoded.transform(features.reshape(-1,1)).toarray()
print(features.shape)

In [ ]:
def encode_string(feature):
    enc=preprocessing.LabelEncoder()
    enc.fit(feature)
    enc_feature=enc.transform(feature)
    ohe=preprocessing.OneHotEncoder()
    encoded=ohe.fit(enc_feature.reshape(-1,1))
    return encoded.transform(enc_feature.reshape(-1,1)).toarray()
columns=['Education',
 'Occupation',
 'Gender',
 'MaritalStatus',]
for col in columns:
    temp=encode_string(aw_custs[col])
    features=np.concatenate([features,temp],axis=1)
print(features.shape)

In [ ]:
features=np.concatenate([features,np.array(aw_custs[['HomeOwnerFlag','NumberCarsOwned','NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'Age']])],axis=1)
features.shape

In [ ]:
aw_aveMonth

In [ ]:
nr.seed(123)
labels=np.array(aw_bike['BikeBuyer'])
index=range(features.shape[0])
index=ms.train_test_split(index,test_size=3000)
x_train=features[index[0],:]
y_train=np.ravel(labels[index[0]])
x_test=features[index[1],:]
y_test=np.ravel(labels[index[1]])

In [ ]:
scaler = preprocessing.StandardScaler().fit(x_train[:,-6:])
x_train[:,-6:]=scaler.transform(x_train[:,-6:])
x_test[:,-6:]=scaler.transform(x_test[:,-6:])
print(x_train.shape)
x_train[:5,:]

In [ ]:
logistic_mod = linear_model.LogisticRegression() 
logistic_mod.fit(x_train, y_train)

In [ ]:
logistic_mod.score(x_test,y_test)

In [ ]:
logistic_mod.predict_proba(x_test)

In [ ]:
nr.seed(1234)
rf_cla = RandomForestClassifier()
rf_cla.fit(x_train, y_train)

In [ ]:
rf_cla.score(x_test, y_test)

In [ ]:
nr.seed(111)
ab_cla=AdaBoostClassifier()
ab_cla.fit(x_train, y_train)

In [ ]:
ab_cla.score(x_test,y_test)

In [ ]:
nr.seed(222)
nn_cla=MLPClassifier()
nn_cla.fit(x_train,y_train)

In [ ]:
nn_cla.score(x_test,y_test)

In [ ]:
nr.seed(555)
svm_cla=svm.LinearSVC()
svm_cla.fit(x_train, y_train)

In [ ]:
svm_cla.score(x_test,y_test)

In [ ]:
nr.seed(555)
svm_cla=svm.SVC()
svm_cla.fit(x_train, y_train)

In [ ]:
svm_cla.score(x_test,y_test)

In [ ]:
nr.seed(666)
inside=ms.KFold(n_splits=10,shuffle=True)
nr.seed(777)
outside=ms.KFold(n_splits=10,shuffle=True)

In [ ]:
param_grid = {"max_features": [2, 3, 5, 10, 15], "min_samples_leaf":[3, 5, 10, 20]}
nr.seed(888)
rf_cla=RandomForestClassifier(class_weight = "balanced")
nr.seed(999)
rf_cla=ms.GridSearchCV(estimator = rf_cla, param_grid = param_grid, 
                      cv = inside,
                      scoring = 'roc_auc',
                      return_train_score = True)
rf_cla.fit(features,labels)

In [ ]:
nr.seed(22)
rf_mod=RandomForestClassifier(class_weight='balanced',
                             max_features=rf_cla.best_estimator_.max_features,
                             min_samples_leaf=rf_cla.best_estimator_.min_samples_leaf)
rf_mod.fit(x_train, y_train)

In [ ]:
rf_mod.score(x_test,y_test)  

In [ ]:
nb_mod =GaussianNB()
nb_mod.fit(x_train,y_train)

In [ ]:
nb_mod.score(x_test,y_test)

In [ ]:
nr.seed(77)
inside=ms.KFold(n_splits=10,shuffle=True)
nr.seed(88)
outside=ms.KFold(n_splits=10,shuffle=True)

In [ ]:
nr.seed(99)
param_grid={"alpha":[0.0001, 0.001, 0.01, 0.1, 1, 10]}
nb_cla=BernoulliNB()
cla=ms.GridSearchCV(estimator = nb_cla, param_grid = param_grid, 
                      cv = inside,
                      scoring = 'roc_auc',
                      return_train_score = True)
cla.fit(features,labels)
cla.best_estimator_.alpha

In [ ]:
nb_mod=BernoulliNB(alpha=cla.best_estimator_.alpha)
nb_mod.fit(x_train,y_train)

In [ ]:
nb_mod.score(x_test,y_test)